In [1]:

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import sys
import time

import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline 
import tensorflow as tf

from cifar10 import CIFAR10

In [2]:
config = tf.ConfigProto()
#config.gpu_options.per_process_gpu_memory_fraction = 0.4
tf.set_random_seed(1)
sess = tf.InteractiveSession(config=config)

In [3]:
# Load dataset
batch_size = 64
cifar10 = CIFAR10(batch_size=batch_size, validation_proportion=0.1, augment_data=False, file='data_original.csv')

SUMMARIES_DIR = './summaries/convnet'

In [4]:
# Model blocks
def conv_layer(input_tensor, kernel_shape, layer_name):
    # input_tensor b01c
    # kernel_shape 01-in-out
    weights = tf.get_variable("weights", kernel_shape,
                               initializer = tf.contrib.layers.xavier_initializer_conv2d())
    biases = tf.get_variable("biases", [kernel_shape[3]],
                             initializer=tf.constant_initializer(0.05))
    
    tf.summary.histogram(layer_name + "/weights", weights)
    tf.summary.histogram(layer_name + "/biases", biases)
    
    # Other options are to use He et. al init. for weights and 0.01 
    # to init. biases.
    conv = tf.nn.conv2d(input_tensor, weights, 
                       strides = [1, 1, 1, 1], padding='SAME')
    return tf.nn.relu(conv + biases)

def fc_layer(input_tensor, weights_shape, layer_name):
    # weights_shape in-out
    weights = tf.get_variable("weights", weights_shape,
                              initializer = tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable("biases", [weights_shape[1]],
                             initializer=tf.constant_initializer(0.0))
    tf.summary.histogram(layer_name + "/weights", weights)
    tf.summary.histogram(layer_name + "/biases", biases)
    mult_out = tf.matmul(input_tensor, weights)
    return tf.nn.relu(mult_out+biases)


In [5]:
# Model
model_input = tf.placeholder(tf.float32, name='model_input', 
                             shape=(batch_size,1,200,1))
tf.summary.image('input', model_input, 10)

keep_prob = tf.placeholder(tf.float32, name='dropout_prob', shape=())

target = tf.placeholder(tf.float32, name='target', shape=(batch_size, 9))

 # Reshape tensor to MLP
first_layer_input = tf.reshape(model_input, [-1,200], name='first_layer_input')
# First layer
layer_name = 'fc1'
with tf.variable_scope(layer_name):
    fc1_out = fc_layer(first_layer_input, [200, 150], layer_name)
fc1_out_drop = tf.nn.dropout(fc1_out, keep_prob)
# Second layer
layer_name = 'fc2'
with tf.variable_scope(layer_name):
    fc2_out = fc_layer(fc1_out_drop, [150, 100],layer_name)
fc2_out_drop = tf.nn.dropout(fc2_out, keep_prob)
#Third Layer
layer_name = 'fc3'
with tf.variable_scope(layer_name):
    fc3_out = fc_layer(fc2_out_drop, [100,50], layer_name)
fc3_out_drop = tf.nn.dropout(fc3_out, keep_prob)

layer_name = 'fc4'
with tf.variable_scope(layer_name):
    fc4_out = fc_layer(fc3_out_drop, [50,10], layer_name)
fc4_out_drop = tf.nn.dropout(fc4_out, keep_prob)

layer_name = 'fc5'
with tf.variable_scope(layer_name):
    fc5_out = fc_layer(fc4_out_drop, [10,9], layer_name)
model_output = fc5_out


In [6]:


with tf.name_scope('loss_function'):
    cross_entropy = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits=model_output, labels=target,
                                           name='cross_entropy'))
    tf.summary.scalar('cross_entropy', cross_entropy)

In [ ]:
# Optimization
with tf.name_scope('optimizer'):
    optimizer = tf.train.AdamOptimizer()
    grads_vars = optimizer.compute_gradients(cross_entropy)
    optimizer.apply_gradients(grads_vars)
    train_step = optimizer.minimize(cross_entropy)

# Metrics
correct_prediction = tf.equal(tf.argmax(model_output, 1),
                             tf.argmax(target, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name='accuracy')
tf.summary.scalar('accuracy', accuracy)

# Useful training functions
def validate():
    batches = cifar10.getValidationSet(asBatches=True)
    accs = []
    for batch in batches:
        data, labels = batch
        acc = sess.run(accuracy,
                       feed_dict={
                model_input: data,
                target: labels,
                keep_prob: 1.0
            })
        accs.append(acc)
    mean_acc = np.array(accs).mean()
    summary = sess.run(merged,
                       feed_dict={
                model_input: data,
                target: labels,
                keep_prob: 1.0
            })
    return summary, mean_acc
def test():
    batches = cifar10.getTestSet(asBatches=True)
    accs = []
    for batch in batches:
        data, labels = batch
        acc = sess.run(accuracy,
                       feed_dict={
                model_input: data,
                target: labels,
                keep_prob: 1.0
            })
        accs.append(acc)
    mean_acc = np.array(accs).mean()
    return mean_acc

In [ ]:
# Train model
merged = tf.summary.merge_all()
train_writer = tf.summary.FileWriter(SUMMARIES_DIR+'/train',
                                     sess.graph)
validation_writer = tf.summary.FileWriter(SUMMARIES_DIR+'/validation')
sess.run(tf.global_variables_initializer())
cifar10.reset()
print("Trainable variables")
for n in tf.trainable_variables():
    print(n.name)

epochs = 200
mean_gradients = np.zeros([len(tf.trainable_variables()), epochs])
std_gradients = np.zeros([len(tf.trainable_variables()), epochs])

t_i = time.time()
n_batches = cifar10.n_batches
while cifar10.getEpoch() < epochs:
    epoch = cifar10.getEpoch()
    batch, batch_idx = cifar10.nextBatch()
    batch_data = batch[0]
    batch_labels = batch[1]
    
    # just a training iteration
    _ = sess.run(train_step,
                feed_dict={
            model_input: batch_data,
            target: batch_labels,
            keep_prob: 0.5
        })
    
    step = batch_idx+epoch*n_batches
    
    # Write training summary
    if step%50==0:
        summary = sess.run(merged,
                          feed_dict={
                model_input: batch_data,
                target: batch_labels,
                keep_prob: 0.5 # set to 1.0 at inference time
            })
        train_writer.add_summary(summary, step)
        
    # gradient (by layer) statistics over last training batch & validation summary
    if batch_idx==0:
        loss, acc, grads = sess.run((cross_entropy, accuracy, grads_vars), 
                      feed_dict={
            model_input: batch_data,
            target: batch_labels,
            keep_prob: 1.0
        })

        for layer in range(len(tf.trainable_variables())):
            mean_gradients[layer, epoch] = np.mean(np.abs(grads[layer][0]))
            std_gradients[layer, epoch] = np.std(np.abs(grads[layer][0]))
        print("Epoch %d, training loss %f, accuracy %f" % (epoch, loss, acc))
        
        summary, validation_accuracy = validate()
        validation_writer.add_summary(summary, step)
        print("Validation accuracy %f" % validation_accuracy)
        print("Time elapsed", (time.time()-t_i)/60.0, "minutes")
train_writer.flush()
validation_writer.flush()
test_acc = test()
print("Testing set accuracy %f" % test_acc)

Trainable variables
fc1/weights:0
fc1/biases:0
fc2/weights:0
fc2/biases:0
fc3/weights:0
fc3/biases:0
fc4/weights:0
fc4/biases:0
fc5/weights:0
fc5/biases:0
Epoch 0, training loss 2.234843, accuracy 0.093750
Validation accuracy 0.066667
Time elapsed 0.0016490379969278971 minutes
Epoch 1, training loss 2.076166, accuracy 0.234375
Validation accuracy 0.282292
Time elapsed 0.008875628312428793 minutes
Epoch 2, training loss 1.985534, accuracy 0.187500
Validation accuracy 0.272917
Time elapsed 0.015715765953063964 minutes
Epoch 3, training loss 1.901565, accuracy 0.296875
Validation accuracy 0.263542
Time elapsed 0.02236416737238566 minutes
Epoch 4, training loss 1.922077, accuracy 0.265625
Validation accuracy 0.271875
Time elapsed 0.029246441523234048 minutes
Epoch 5, training loss 1.912251, accuracy 0.312500
Validation accuracy 0.280208
Time elapsed 0.03582298755645752 minutes
Epoch 6, training loss 1.794742, accuracy 0.312500
Validation accuracy 0.277083
Time elapsed 0.042800295352935794 

Epoch 67, training loss 1.322944, accuracy 0.515625
Validation accuracy 0.485417
Time elapsed 0.4709646900494893 minutes
Epoch 68, training loss 1.460385, accuracy 0.453125
Validation accuracy 0.481250
Time elapsed 0.47767444054285685 minutes
Epoch 69, training loss 1.337243, accuracy 0.453125
Validation accuracy 0.467708
Time elapsed 0.4840227007865906 minutes
Epoch 70, training loss 1.331046, accuracy 0.500000
Validation accuracy 0.483333
Time elapsed 0.49065067768096926 minutes
Epoch 71, training loss 1.435654, accuracy 0.468750
Validation accuracy 0.490625
Time elapsed 0.4970617810885111 minutes
Epoch 72, training loss 1.300237, accuracy 0.531250
Validation accuracy 0.460417
Time elapsed 0.5044920603434245 minutes
Epoch 73, training loss 1.383091, accuracy 0.500000
Validation accuracy 0.490625
Time elapsed 0.5122857411702474 minutes
Epoch 74, training loss 1.111003, accuracy 0.609375
Validation accuracy 0.456250
Time elapsed 0.5204034805297851 minutes
Epoch 75, training loss 1.4521

In [ ]:
# Plotting gradients
n_layers = len(tf.trainable_variables()) // 2
x = np.arange(epochs)
i = 0
plt.figure()
while i < n_layers*2:
    plt.errorbar(x,mean_gradients[i,:],std_gradients[i,:])
    i = i + 2
plt.yscale('log')
plt.ylabel('Gradient')
plt.xlabel('Epochs')
plt.title('Weights Gradient by Layer')
plt.legend(["conv1","conv2","fc1","fc2"][-n_layers:])
plt.xlim(-0.2, epochs)
plt.show()
i = 1
plt.figure()
while i < n_layers*2:
    plt.errorbar(x,mean_gradients[i,:],std_gradients[i,:])
    i = i + 2
plt.yscale('log')
plt.ylabel('Gradient')
plt.xlabel('Epochs')
plt.title('Biases Gradient by Layer')
plt.legend(["conv1","conv2","fc1","fc2"][-n_layers:])
plt.xlim(-0.2, epochs)
plt.show()

In [ ]:
def confusion_matrix(targets, outputs):
    '''Returns a confusion matrix. Both targets and outputs
    should be 1-D arrays of zeros and ones.'''
    encoded_data = 2*targets+outputs  # Map targets and outputs to {0, 1, 2, 3}
    TN = np.sum(encoded_data == 0)  # True negatives
    FP = np.sum(encoded_data == 1)  # False positives
    FN = np.sum(encoded_data == 2)  # False negatives
    TP = np.sum(encoded_data == 3)  # True positives
    return ((TP, FP), (FN, TN))

def roc_curve(targets, outputs):
    '''Returns a ROC curve. Outputs should be in range 0-1
    in order to move the threshold.'''
    tpr = []
    fpr = []
    for threshold in np.linspace(0, 1, 1000):
        outputs_with_threshold = (outputs > threshold).astype(np.float)
        ((TP, FP), (FN, TN)) = confusion_matrix(
            targets, 
            outputs_with_threshold)
        tpr.append(TP/(TP+FN))
        fpr.append(FP/(FP+TN))
    return np.array(tpr), np.array(fpr)

accs = sess.run(accuracy,
             feed_dict={
            model_input: cifar10.training_data,
                       target: cifar10.training_labels,
            keep_prob: 1.0
        })

accs = np.asarray(accs)
[[VN,FP],[FN,VP]] = confusion_matrix(
    accs,
    (accs>0.5).astype(np.float))
print('VP: %d, VN: %d, FP: %d, FN: %d' %(VP,VN,FP,FN))
print('Porcentaje de clasificaciones correctas: %%%f' %(100.0*(VP+VN)/(VP+VN+FP+FN)))
print('Precision: %%%f' %(100.0*VP/(VP+FP)))
print('Recall: %%%f' %(100.0*VP/(VP+FN)))
print('')
